In [1]:
import pandas as pd
import numpy as np
import pyodbc
import os
import requests

In [2]:
import pyodbc
import urllib
import sqlalchemy
# MySQL connection parameters
db_params = {
    'user': 'han',
    'password': 'joseph',
    'host': '127.0.0.1',  # Usually 'localhost' or an IP address
    'port': '4450',  # Default MySQL port is 3306
    'database': 'ITRAVEL',
}

# URL-encode the password (if it contains special characters)
db_params['password'] = urllib.parse.quote_plus(db_params['password'])

# Create a MySQL database connection using SQLAlchemy
params_mysql = "mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}".format(**db_params)
engine_mysql = sqlalchemy.create_engine(params_mysql)

In [3]:
engine_p=engine_mysql

In [14]:
'''get locations data from source database'''
df_locations = pd.read_sql_table('locations',engine_p)
df_reviews  = pd.read_sql_table('location_reviews',engine_p)
df_details  = pd.read_sql_table('location_details',engine_p)

In [15]:
df_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4272 entries, 0 to 4271
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       4272 non-null   int64 
 1   location_id              4272 non-null   int64 
 2   name                     4272 non-null   object
 3   description              2688 non-null   object
 4   web_url                  4272 non-null   object
 5   address_obj              4272 non-null   object
 6   ancestors                4272 non-null   object
 7   latitude                 4272 non-null   object
 8   longitude                4272 non-null   object
 9   timezone                 4272 non-null   object
 10  email                    1997 non-null   object
 11  phone                    3325 non-null   object
 12  website                  2588 non-null   object
 13  write_review             4272 non-null   object
 14  ranking_data             3545 non-null  

In [6]:
# Define a location details function
def call_location_details(location_id):
    """This function get location details through api ."""
    
    # API URL
    api_url = "https://api.content.tripadvisor.com/api/v1/location/" + str(location_id) + "/details?language=en&currency=USD&key=72577774A72F48D9BF1180DEA12E312A"
  
    # Make API request
    response = requests.get(api_url)
    data = response.json()

#     df = pd.DataFrame(data)

    return data

In [7]:
def dict2df(df):
    for key, value in df.items():
        df[key]=str(value)

    try:
        df = pd.DataFrame(df, index=[0])  
        df['location_id'] = df['location_id'].str.extract('(\d+)').astype(int)
        df['num_reviews'] = df['num_reviews'].str.extract('(\d+)').astype(int)
        df['photo_count'] = df['photo_count'].str.extract('(\d+)').astype(int)
    except KeyError:
        return None
    
    
    return df

In [8]:
def get_category(df):
    df_category=pd.DataFrame()
    # Loop through the rows using iterrows()
    for index, row in df.iterrows():
        data=eval(row['category'])
        
        category_name = data.get('name', '')
        category_localized_name = data.get('localized_name', '')
        
        df1 = pd.DataFrame({'category_name': category_name, 
                            'category_localized_name': category_localized_name
                           }, index=[0])
        df_category=pd.concat([df_category,df1])

#         if index >2 :
#             break
    df_category=df_category.reset_index(drop=True)
   
    return df_category

In [16]:
# Loop through DataFrame rows

df_locations_new=df_locations[~df_locations['location_id'].isin(df_details['location_id'])]

df_locations_new = df_locations_new.reset_index(drop=True)
# df_locations_new

In [17]:
df_locations_new

,id,location_id,name,distance,bearing,address_obj
0,2658,294013,Abu Dhabi,0.19947230061959376,north,"{'street1': '', 'street2': '', 'state': 'Emira..."
1,2682,2630227,Emirate of Abu Dhabi,0.8894672196257867,north,"{'street1': '', 'street2': '', 'country': 'Uni..."
2,3291,8838574,Al Aqah,0.023002469564216317,north,"{'state': 'Emirate of Fujairah', 'country': 'U..."
3,3400,2085520,Al Jazirat Al Hamra,1.6354098186019919,northwest,"{'street1': '', 'street2': '', 'state': 'Emira..."
4,3551,1028283,Dibba Al Fujairah,1.2118805923469316,northeast,"{'street1': '', 'street2': '', 'state': 'Emira..."
5,3666,2630267,Dibba Al Hisn,0.09518342614280281,north,"{'state': 'Emirate of Sharjah', 'country': 'Un..."
6,3780,311321,Fujairah,0.7950724782437748,west,"{'street1': '', 'street2': '', 'state': 'Emira..."
7,3935,672742,Jebel Ali,2.9001229733608898,east,"{'street1': '', 'street2': '', 'state': 'Emira..."
8,4057,1137990,Kalba,1.6675253769629128,south,"{'street1': '', 'street2': '', 'state': 'Emira..."
9,4119,660713,Khorfakkan,1.230560155452929,north,"{'street1': '', 'street2': '', 'country': 'Uni..."


In [18]:
# Get location detail data from API

df_new_details=pd.DataFrame()
for index, row in df_locations_new.iterrows():
    location_id = row['location_id']
        
    dt=call_location_details(location_id)

    df1=dict2df(dt)
    if df1 is not None:
        df_new_details=pd.concat([df_new_details, df1], ignore_index=True)    
    
#     if index > 200:
#         break


In [22]:
df_new_details

""


In [ ]:
# Add column: category_name,category_localized_name 

df1=get_category(df_new_details)

df_new_details['category_name']=df1['category_name']
df_new_details['category_localized_name']=df1['category_localized_name']
# df_details['category']
# df_new_details[['category','category_name','category_localized_name']]
# df_details[['category'],['category_name'],['category_localized_name']]

# df_details = df_details.drop('id', axis=1)
df_new_details

In [13]:
# Save new location Detail data to talbe location_details.
df_new_details.to_sql("location_details",engine_p,if_exists='append',index=False)

1505